# Paso a paso de la ejecucion del script
En este nootebook se irán explicando paso a paso las partes creadas para que funcine nuestro código

In [1]:
import pandas as pd
import numpy as np
import re
from joblib import load, dump
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV, TimeSeriesSplit
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.compose import TransformedTargetRegressor

## Lectura del dataset

In [2]:
# Lectura del dataset
fullDataset = pd.read_csv('../datasets/Sotavento/data_target_stv_2016_2017.csv', index_col=0, parse_dates=True)
x_col, target_col = fullDataset.columns[:-1], fullDataset.columns[-1]
data, target = fullDataset[x_col], fullDataset[target_col]

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/Sotavento/data_target_2016_2017.csv'

In [4]:
fullDataset

,"10u_(44.0, -9.5)","10u_(44.0, -9.375)","10u_(44.0, -9.25)","10u_(44.0, -9.125)","10u_(44.0, -9.0)","10u_(44.0, -8.875)","10u_(44.0, -8.75)","10u_(44.0, -8.625)","10u_(44.0, -8.5)","10u_(44.0, -8.375)",...,"vel100_(43.0, -6.0)","vel100_(43.125, -6.0)","vel100_(43.25, -6.0)","vel100_(43.375, -6.0)","vel100_(43.5, -6.0)","vel100_(43.625, -6.0)","vel100_(43.75, -6.0)","vel100_(43.875, -6.0)","vel100_(44.0, -6.0)",targ
prediction date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,1.577097,1.675974,1.836863,1.821115,1.681467,1.424021,1.190012,1.106394,1.108713,1.304880,...,5.321404,5.971150,5.862873,5.164530,5.391425,6.107658,6.398007,6.472929,6.793221,0.235006
2016-01-01 01:00:00,1.320722,1.059857,1.200238,1.276654,1.713910,1.246137,0.725141,0.203412,0.215253,0.447186,...,5.435709,6.092992,5.361253,3.485183,3.988883,4.845162,5.260955,5.464975,5.908251,0.323947
2016-01-01 02:00:00,1.562336,1.023518,0.415607,0.335896,0.563801,0.696369,0.487873,0.106770,-0.157024,-0.133709,...,5.325067,6.018884,4.991258,2.281320,2.807106,3.654994,4.170403,4.632105,5.078648,0.440889
2016-01-01 03:00:00,0.198002,0.025639,0.332890,-0.118526,-0.533199,-1.093990,-1.123653,-1.040646,-0.928341,-0.824581,...,5.177444,5.716941,4.493564,1.185751,1.837592,3.329041,4.260316,4.934864,5.643699,0.436247
2016-01-01 04:00:00,1.604583,1.043792,1.073821,0.065032,-0.961823,-1.394685,-1.394074,-1.230012,-1.500520,-1.470124,...,5.317643,5.648662,4.902998,3.442938,3.724846,4.111763,4.328406,4.885375,5.730442,0.554128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,13.205357,13.218540,13.285679,13.400913,13.492222,13.571568,13.746616,13.927280,14.026646,14.026402,...,6.993787,6.961161,8.223429,9.236624,8.309718,11.641448,17.365959,19.244395,19.069818,0.589443
2017-12-31 20:00:00,13.454221,13.537229,13.584592,13.700315,13.778440,13.636838,13.510373,13.691525,13.993772,14.166135,...,7.092767,6.628419,7.578160,8.436968,8.207733,10.873664,15.504934,18.109813,18.008255,0.570834
2017-12-31 21:00:00,12.977776,13.069085,13.196038,13.437737,13.550042,13.434808,13.520745,13.726800,13.812249,13.859124,...,7.058320,6.408309,7.045037,8.654056,8.371027,10.311946,13.318638,16.395524,18.839948,0.587978


## Parte optativa de reducción de dimension
Esta parte es para las pruebas iniciales, luego se debe omitir, puesto que estamos reduciendo el dataset

In [5]:
### En este caso el dataset ya está en el intervalo [0-1]
### En caso de que no lo estuviera, dividimos entre la potencia máxima del parque
 
#pot_install = 17560
#target = target / pot_install
### Valores desde los que crear la nueva región
midLat = 43.375
midLon = -7.875
jump = 0.125
n = 5
### Latitud y longitud
listLat = list(np.arange(midLat-jump*n, midLat+jump*(n+1), jump))
listLon = list(np.arange(midLon-jump*n, midLon+jump*(n+1), jump))
search_coord = []
for lat in listLat:
    for lon in listLon:
        search_coord.append(f'({lat}, {lon})')
### Con una expresión regular limpiamos las columnas que no nos interesna
col = list(data.columns)
new_coord = []
for coord in search_coord:
    r = re.compile(f".*{coord}")
    new_coord += list(filter(r.match, col))
newDataframe = data[new_coord]
newDataframe.index = data.index
data = newDataframe

## Partes a incluir en el pipeline

In [ ]:
# Creación de PCA sin parámetros para luego hiperparametrizar
pca = PCA()
# Creación del modelo de regresion final
reg = Ridge(max_iter=10000)

## Creación del Pipeline y estandarización del target

In [ ]:
# Pipeline
regr_base = Pipeline(steps=[('std_sc', StandardScaler()),
                            ('pca', pca),
				            ('reg', reg)])

# Estandarizamos el target también
y_transformer = StandardScaler()
inner_estimator = TransformedTargetRegressor(regressor=regr_base, transformer=y_transformer)

## Hiperparametrización

In [ ]:
# Parametros a hiperparametrizar
l_alpha = [10.**k for k in range(-3, 5)]
n_components = list(range(50, 301, 25))
param_grid = {
    'regressor__reg__alpha': l_alpha,
    'regressor__pca__n_components': n_components,
}
# Numero de splits para Cross Validation
n_splits = 2
kf = KFold(n_splits, shuffle=False)
# Hiperparametrizacion
cv_estimator = GridSearchCV(estimator=inner_estimator, 
                            param_grid=param_grid, 
                            cv=kf, 
                            scoring='neg_mean_absolute_error', 
                            return_train_score=True, 
                            n_jobs=5, 
                            verbose=1)

## Entrenamiento y volcado de datos

In [ ]:
# Fit datos 
_ = cv_estimator.fit(data.values, target.values)
print(f"Mejor estimador: {cv_estimator.best_estimator_}")
print(f"Mejor puntuacion - mejor estimador: {cv_estimator.best_score_}")
# Guardar datos en fichero
filename = f'cv_estimator__ridge_{n_components[0]}_{n_components[-1]}_{n_components[1]-n_components[0]}'
dump(cv_estimator, filename)